# Capítulo 5: Máquinas vectoriales de soporte (SVM)

</br></br>
Una máquina de vectores de soporte (SVM) es un modelo de aprendizaje automático potente y versátil, capaz de realizar la clasificación lineal o no lineal, la regresión e incluso la detección de novedades. Los SVM brillan con conjuntos de datos no lineales de tamaño pequeño a mediano (es decir, de cientos a miles de instancias), especialmente para tareas de clasificación. Sin embargo, no se escalan muy bien a conjuntos de datos muy grandes, como verás.

Este capítulo explicará los conceptos básicos de los SVM, cómo usarlos y cómo funcionan. ¡Vamos a entrar!

## Clasificación lineal de SVM

</br></br>
La idea fundamental detrás de los SVM se explica mejor con algunas imágenes. 
La Figura 5-1 muestra parte del conjunto de datos del iris que se introdujo al final del capítulo 4. Las dos clases se pueden separar fácilmente con una línea recta (son separables linealmente). 
El gráfico de la izquierda muestra los límites de decisión de tres posibles clasificadores lineales. El modelo cuyo límite de decisión está representado por la línea discontinua es tan malo que ni siquiera separa las clases correctamente. 

Los otros dos modelos funcionan perfectamente en este conjunto de entrenamiento, pero sus límites de decisión se acercan tanto a las instancias que estos modelos probablemente no funcionarán tan bien en las nuevas instancias. Por el contrario, la línea continua en el gráfico de la derecha representa el límite de decisión de un clasificador SVM; esta línea no solo separa las dos clases, sino que también se mantiene lo más lejos posible de las instancias de entrenamiento más cercanas. 

Se puede pensar en un clasificador SVM que se ajusta a la calle más ancha posible (representada por las líneas discontinuas paralelas) entre las clases. Esto se llama clasificación de gran margen.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0501.png)

(_Figura 5-1. Clasificación de gran margen_)

Tenga en cuenta que agregar más instancias de entrenamiento "fuera de la calle" no afectará en absoluto al límite de la decisión: está totalmente determinado (o "apoyado") por las instancias ubicadas en el borde de la calle. Estas instancias se llaman vectores de soporte (están rodeados en la Figura 5-1).